In [1]:
import sys, pygame
from pygame import Color, surfarray
import numpy as np
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [89]:
cell_width = 90
cell_height = 90

cell_size = 10 # Number of pixels per cell

display_width = cell_width*cell_size
display_height = cell_height*cell_size

color_dict = {
        0: (255, 255, 255),
        1: (0, 0, 0)
        }

n_bits = 3

EVOLVE_COLOR = True

In [20]:
def init_states(itype='random', return_btm=False):
    states = [[0]*cell_width for i in range(cell_height)] # Zero State
    if itype == 'random':
        states[0] = [random.randrange(2) for i in range(cell_width)] # Random State
    if return_btm:
        return states[0]
    return states

In [21]:
def next_gen(left, middle, right, nth_rule):
    left, middle, right = int(bool(left)), int(bool(middle)), int(bool(right))
    value = int(str(left) + str(middle) + str(right), 2)
    return get_rule(nth_rule)[value]

def new_state(states, nth_rule=30):
    new = states[0].copy() # Get bottom row
    
    if nth_rule == 'random':
        nth_rule = random.randrange(2**(2**n_bits))
    
    new[1:-1] = [next_gen(new[i-1], new[i], new[i+1], nth_rule) for i in range(1, len(new)-1)] # New row, keep edge values
    states.insert(0, new) # Add new row to state array
    states.pop() # Shift states up 1 row
    
    return states # return state array

"""
    Returns PixelArray from state list
"""
def states_to_pix(states):
    return [[color_dict[c] for c in s] for s in states]

In [22]:
def get_rules(n_bits=3, nth_rule=None):
    n_states = 2**n_bits
    rules = np.zeros((2**n_states, n_states))
    for i in range(2**n_states):
        rule = format(i, '0'+str(n_states)+'b')
        rules[i,:] = np.array([int(b) for b in rule])
        
    if nth_rule != None:
        return rules[nth_rule, :]
    
    return rules

def get_rule(nth_rule, n_bits=3):
    n_states = 2**n_bits
    rule = format(nth_rule, '0'+str(n_states)+'b')
    return np.array([int(b) for b in rule])

In [83]:
def update_color_dict(color='random', max_delta=10):
    if color == 'random':
        rgb = random.randrange(3) # Random Channel
        delta = random.randrange(-max_delta, max_delta) # Random change in color
        
        channels = list(color_dict[1])
        channel = channels[rgb]     
        channel += delta
        channel = channel if channel >= 0 else 0
        channel = channel if channel <= 255 else 255
        channels[rgb] = channel
        color_dict[1] = tuple(channels)

In [92]:
states = init_states()

pygame.init()

screen = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('1D Cellular Automata')

#create a surface with the size as the array
surface = pygame.Surface((cell_width, cell_height))
surface = pygame.transform.scale(surface, (display_width, display_height)) # Scale Size Up


run = True
cnt = 0
nth_rule = 0
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False
            #sys.exit()

    pygame.time.delay(5)
    
    # Reset bottom row if limited complexity
    if sum(states[0])/len(states[0]) < 0.05:
        states[0] = init_states(return_btm=True)
    
    if cnt % cell_height == cell_height-1:
        nth_rule += 1
        print('Rule:', nth_rule)
    
    # Scale Screen Size Down to edit pixel array at cell size=1 --------------------------------------------------
    surface = pygame.transform.scale(surface, (cell_width, cell_height))
    px = pygame.PixelArray(surface)
    
    # Translate States to Surface as RGB Colors
    states = new_state(states, nth_rule=nth_rule)
    new_px = states_to_pix(states)
    for i, r in enumerate(new_px):
        px[:,i] = new_px[len(new_px) - 1 - i]

    px.close()
    
    # Scale Screen Size Up to display pixel array at cell size=multiplier ----------------------------------------
    surface = pygame.transform.scale(surface, (display_width, display_height))  # Scale Size Up     
    screen.blit(surface, (0, 0)) # Update all pixels on screen object
    
    pygame.display.update()
    
    if EVOLVE_COLOR:
        update_color_dict()
    cnt += 1
    
pygame.quit()

Rule: 1
Rule: 2
Rule: 3
Rule: 4
